In [1]:
import pandas as pd
import numpy as np
import datetime
from time import ctime
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Datos de entrenamiento limpios y con features agregados
train = pd.read_csv('../data/to_train_01.csv', low_memory=False, lineterminator='\n')

print(train.info())
train.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 85 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   building_id                             260601 non-null  int64  
 1   damage_grade                            260601 non-null  int64  
 2   geo_level_1_id                          260601 non-null  int64  
 3   geo_level_2_id                          260601 non-null  int64  
 4   geo_level_3_id                          260601 non-null  int64  
 5   count_floors_pre_eq                     260601 non-null  int64  
 6   age                                     260601 non-null  int64  
 7   area_percentage                         260601 non-null  int64  
 8   height_percentage                       260601 non-null  int64  
 9   has_superstructure_adobe_mud            260601 non-null  bool   
 10  has_superstructure_mud_mortar_stone     2606

,building_id,damage_grade,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,anio_edificacion,construida_antes_de_codigo,cant_pisos_por_antiguedad,cant_pisos_por_area,cant_pisos_por_altura,area_por_antiguedad,altura_por_antiguedad,area_por_altura,producto_area_altura,area_al_cuadrado,altura_al_cuadrado,promedio_area_altura,cant_familias_por_pisos,cant_familias_por_area,cant_familias_por_altura,cant_materiales,land_surface_condition_t,land_surface_condition_o,land_surface_condition_n,plan_configuration_d,plan_configuration_u,plan_configuration_s,plan_configuration_q,plan_configuration_m,plan_configuration_c,plan_configuration_a,plan_configuration_n,plan_configuration_f,plan_configuration_o,roof_type_n,roof_type_q,roof_type_x,other_floor_type_q,other_floor_type_x,other_floor_type_j,other_floor_type_s,foundation_type_r,foundation_type_w,foundation_type_i,foundation_type_u,foundation_type_h,legal_ownership_status_v,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_w,ground_floor_type_f,ground_floor_type_x,ground_floor_type_v,ground_floor_type_z,ground_floor_type_m,position_t,position_s,position_j,position_o
0,802906,3,6,487,12198,2,30,6,5,True,True,False,False,False,False,False,False,False,False,False,1,False,False,False,False,False,False,False,False,False,False,1985,False,0.066335,0.333333,0.4,0.199005,0.165837,1.2,30,36,25,5.5,0.5,0.166667,0.2,2,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0


In [3]:
# Datos de prueba
test = pd.read_csv('../data/to_test_01.csv', low_memory=False, lineterminator='\n')
print(test.info())
test.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86868 entries, 0 to 86867
Data columns (total 84 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   building_id                             86868 non-null  int64  
 1   geo_level_1_id                          86868 non-null  int64  
 2   geo_level_2_id                          86868 non-null  int64  
 3   geo_level_3_id                          86868 non-null  int64  
 4   count_floors_pre_eq                     86868 non-null  int64  
 5   age                                     86868 non-null  int64  
 6   area_percentage                         86868 non-null  int64  
 7   height_percentage                       86868 non-null  int64  
 8   has_superstructure_adobe_mud            86868 non-null  int64  
 9   has_superstructure_mud_mortar_stone     86868 non-null  int64  
 10  has_superstructure_stone_flag           86868 non-null  in

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_t,land_surface_condition_o,land_surface_condition_n,plan_configuration_d,plan_configuration_u,plan_configuration_s,plan_configuration_q,plan_configuration_m,plan_configuration_c,plan_configuration_a,plan_configuration_n,plan_configuration_f,plan_configuration_o,roof_type_n,roof_type_q,roof_type_x,other_floor_type_q,other_floor_type_x,other_floor_type_j,other_floor_type_s,foundation_type_r,foundation_type_w,foundation_type_i,foundation_type_u,foundation_type_h,legal_ownership_status_v,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_w,ground_floor_type_f,ground_floor_type_x,ground_floor_type_v,ground_floor_type_z,ground_floor_type_m,position_t,position_s,position_j,position_o,anio_edificacion,construida_antes_de_codigo,cant_pisos_por_antiguedad,cant_pisos_por_area,cant_pisos_por_altura,area_por_antiguedad,altura_por_antiguedad,area_por_altura,producto_area_altura,area_al_cuadrado,altura_al_cuadrado,promedio_area_altura,cant_familias_por_pisos,cant_familias_por_area,cant_familias_por_altura,cant_materiales
0,300051,17,596,11307,3,20,7,6,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1995,True,0.148883,0.428571,0.5,0.347395,0.297767,1.166667,42,49,36,6.5,0.333333,0.142857,0.166667,1


## Entrenamiento del modelo

In [4]:
features_to_remove = ['building_id', 'geo_level_2_id', 'geo_level_3_id', 'damage_grade']
Y = train['damage_grade']
X = train.drop(features_to_remove, axis=1)

In [5]:
from sklearn import ensemble
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195450 entries, 144109 to 85237
Data columns (total 81 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   geo_level_1_id                          195450 non-null  int64  
 1   count_floors_pre_eq                     195450 non-null  int64  
 2   age                                     195450 non-null  int64  
 3   area_percentage                         195450 non-null  int64  
 4   height_percentage                       195450 non-null  int64  
 5   has_superstructure_adobe_mud            195450 non-null  bool   
 6   has_superstructure_mud_mortar_stone     195450 non-null  bool   
 7   has_superstructure_stone_flag           195450 non-null  bool   
 8   has_superstructure_cement_mortar_stone  195450 non-null  bool   
 9   has_superstructure_mud_mortar_brick     195450 non-null  bool   
 10  has_superstructure_cement_mortar_brick  

### Parameter tuning

> https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

In [ ]:
best_params = {}

param_search = {
    'learning_rate': [0.1],
    'objective':['multiclass'],
    'num_class':[3],
    'num_leaves': [40, 50],
    'metric':['multi_logloss'],
    'n_estimators': [100, 200, 500, 1000],
    'min_child_samples': [20, 40],
    'max_bin': [500],
    'boosting_type' : ['gbdt'],
    'random_state' : [42],
    'colsample_bytree' : [0.5, 0.6, 0.7],
    'subsample' : [0.5, 0.6, 0.7],
    'n_jobs':[-1]
    }


model = GridSearchCV(estimator = LGBMClassifier(), param_grid=param_search, scoring='f1_micro', cv=4, n_jobs=-1, verbose=5)
#model = LGBMClassifier(**best_params)
print('Hora antes de empezar a tunear/entrenar: {}'.format(ctime()))
model.fit(X_train, y_train)
print('Hora al finalizar de tunear/entrenar: {}'.format(ctime()))

Hora antes de empezar a tunear/entrenar: Tue Jul  6 01:12:01 2021
Fitting 4 folds for each of 144 candidates, totalling 576 fits


In [ ]:
print('El mejor score de LightGBM con GridSearchCV fue: {}'.format((model.best_score_).round(4)))
print('Mejor parametros hasta el momento: ')
model.best_params_

In [ ]:
from sklearn.metrics import f1_micro

y_pred = model.predict(X_test)

model_f1_micro = f1_micro(y_test, y_pred)
print('LightGBM F1-micro-score: %.4f' % model_mae)

### Feature importance

In [ ]:
import matplotlib.pyplot as plt

feature_cols = X.columns.tolist()
features = pd.Series(data=model.feature_importances_, index=feature_cols, name='Feature importance')
features.sort_values(ascending=True, inplace=True)
plt.figure(figsize=(10, 10))
features.plot(kind='barh')
plt.xlabel('Importancia')
plt.ylabel('Features')
plt.title('Importancia Features con LightGBM')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred):
    names = sorted(set(y_true))
    cm = confusion_matrix(y_true, y_pred, names)
    df_cm = pd.DataFrame(cm, names, names)

    plt.figure(dpi=100)
    plt.title("Matriz de confusion")
    sns.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='g', square=True)
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.show()